# 제2장 기본적인 추천 시스템
## 2.1 데이터 읽기

In [1]:
import pandas as pd
from nltk import accuracy

u_cols = ['user_id','age','sex','occupation','zip_code']
users = pd.read_csv('data/u.user',sep='|',names=u_cols,encoding='latin-1') # 구분자 |, 열이름 지정
users = users.set_index('user_id') #users의 user_id를 인덱스로 설정
users.head()

,age,sex,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


In [2]:
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 
          'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 
          'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('data/u.item',sep='|',names=i_cols,encoding='latin-1')
movies=movies.set_index('movie_id')
movies.head()

,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [3]:
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('data/u.data',sep='\t',names=r_cols,encoding='latin-1')
ratings = ratings.set_index('user_id')
ratings.head()

,movie_id,rating,timestamp
user_id,,,
196,242,3,881250949
186,302,3,891717742
22,377,1,878887116
244,51,2,880606923
166,346,1,886397596


## 2.2 인기제품 방식

개별 사용자에 대한 정보가 없는 경우나, 정확도에 관계없이 가장 간단한 추천을 제공해야 하는 상황에서 사용하는 방법  
모든 사람에게 똑같은 추천을 하는 방 

In [4]:
# Best-seller 추천
def recom_movie(n_items):
    movie_sort = movie_mean.sort_values(ascending=False)[:n_items]
    # sort_values(ascending=False) 데이터 값을 내림차순으로 정렬
    recom_movies = movies.loc[movie_sort.index] # loc는 인덱싱 할 때 사용(인덱스 이름,열 이름,행 이름)
    recommendations = recom_movies['title']
    return recommendations

# 한줄로 정#
def recom_movie2(n_items):
    return movies.loc[movie_mean.sort_values(ascending=False)[:n_items].index]['title']

movie_mean = ratings.groupby(['movie_id'])['rating'].mean()
recom_movie(5)

movie_id
814                         Great Day in Harlem, A (1994)
1599                        Someone Else's America (1995)
1201           Marlene Dietrich: Shadow and Light (1996) 
1122                       They Made Me a Criminal (1939)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object

## 2.3 추천 시스템의 정확도 측정

RMSE(Root Mean Squared Error)
- 예측값과 해당 실제값의 차이(error)의 제곱(squared)의 평균(mean)의 제곱근(root)

- $RMSE = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}$
    - $y_i$: 실제값 (Ground Truth)  
    - $\hat{y}_i$: 예측값 (Predicted Value)  
    - $n$: 데이터 포인트의 개수  



In [8]:
import numpy as np
# 정확도 계산
def RMSE(y_true,y_pred):
    return np.sqrt(np.mean((np.array(y_true)-np.array(y_pred))**2))

rmse = []
for user in set(ratings.index): # set()은 중복을 제거하고 고유한 값을 가지는 데이터 구조를 생성 (user를 뽑기위해)
    y_true = ratings.loc[user]['rating']
    y_pred = movie_mean[ratings.loc[user]['movie_id']] # user가 평가한 영화들의 평균
    accuracy = RMSE(y_true,y_pred)
    rmse.append(accuracy)
print(np.mean(rmse))

0.996007224010567
